In [ ]:
import numpy as np

In [ ]:
import optimization.optim_tools as optim_tools

kwargs = optim_tools.bisect_max(0, 1, None, None, None,
           bisection_tol=1e-3, solver=None, bisect_verbose=False, max_iters=1100, warm_start=True)
kwargs['max_iters'] = 2000
kwargs

In [ ]:
def svd_inverse(A):
    """ Inverts a matrix using singular value decomposition (SVD).
        The matrix doesn't have to be square or of full rank.
    """
    # A = U*(I*s)*V => is easier to invert (see literature)
    U, s, V = np.linalg.svd(A, full_matrices=False)  # SVD decomposition of A
    for i in xrange(0, len(s)):
        if s[i] > 0.001:
            s[i] = 1/s[i]        # ToDo: damping near singularities
    return np.dot(np.dot(V.T, np.diag(s)), U.T)

In [ ]:
def svd_solve(A, b):
    U, s, V = np.linalg.svd(A, full_matrices=False)  # SVD decomposition of A
    for i in xrange(0, len(s)):
        if s[i] > 0.001:
            s[i] = 1/s[i]        # ToDo: damping near singularities
    return V.T.dot(np.diag(s).dot(U.T.dot(b)))

In [ ]:
def svd_solve2(A, b):
    U, s, V = np.linalg.svd(A, full_matrices=False)  # SVD decomposition of A
    s[s > 0.001] = 1/s[s > 0.001]
    return V.T.dot(np.diag(s).dot(U.T.dot(b)))

In [ ]:
n=6
m=7
A_m = np.random.rand(n,m)
y_m = np.random.rand(n)
np.random.seed(0)

In [ ]:
%%timeit

A_m_inv = svd_inverse(A_m)
dq = np.dot(A_m_inv, y_m)

In [ ]:
%%timeit

dq = svd_solve(A_m, y_m)

In [ ]:
%%timeit

dq = svd_solve2(A_m, y_m)

In [ ]:
#import numpy as np
#import control as con
#from numpy import linalg as LA

import cvxpy
#import optimization.optim_tools as optim_tools#own file with helper

In [ ]:
import rospy
from tf import transformations as trans

R0 = trans.quaternion_matrix([0.500, -0.500, -0.500, -0.500])
t0 = trans.translation_matrix((-0.187, 0.359, 0.403))
T0 = trans.concatenate_matrices(t0, R0)

R1 = trans.quaternion_matrix([-0.323, 0.323, 0.629, 0.629])
t1 = trans.translation_matrix((-0.187, 0.330, 0.393))
T1 = trans.concatenate_matrices(t1, R1)

R2 = trans.quaternion_matrix([-0.208, 0.406, 0.792, 0.406])
t2 = trans.translation_matrix((-0.158, 0.335, 0.386))
T2 = trans.concatenate_matrices(t2, R2)

print T0
print T1
print T2

In [ ]:
import numpy as np
import cvxpy

n = 4

T_tool = cvxpy.Semidef(n, n) # Implys (459) (semidefinite for numerical reasons?)
T_obj = cvxpy.Semidef(n, n) # Implys (459) (semidefinite for numerical reasons?)
T_err0 = cvxpy.Semidef(n, n) # Implys (459) (semidefinite for numerical reasons?)
T_err1 = cvxpy.Semidef(n, n) # Implys (459) (semidefinite for numerical reasons?)
T_err2 = cvxpy.Semidef(n, n) # Implys (459) (semidefinite for numerical reasons?)

constraints = [T0*T_tool == T_obj,
               T1*T_tool == T_obj,
               T2*T_tool == T_obj,
              ]

In [ ]:

# Objective representation
obj = cvxpy.Minimize(cvxpy.trace(T_obj)) # Identical to geo_mean (in term of convexity and result)
#obj = cvxpy.Maximize(cvxpy.log_det(Q)) # Identical to geo_mean (in term of convexity and result)
#obj = cvxpy.Minimize(-cvxpy.geo_mean(Q)) # Not yet implemented

prob = cvxpy.Problem(obj, constraints)
prob.solve(verbose=True)

In [ ]:
import sympy as sp
import numpy as np
import numpy.linalg as la

w = sp.symbols('w_1')
s = sp.symbols('s_1')
b = sp.symbols('b_1:3')
d = sp.symbols('d_1')
x = sp.symbols('x_1:3')
c = sp.symbols('c_1:3')

u = sp.symbols('u_1')

X = sp.Matrix([x]).T
U = sp.Matrix([u])


A = sp.Matrix([[s, w],[-w,s]])
B = sp.Matrix([b]).T
C = sp.Matrix([c])
D = sp.Matrix([d])

AA = sp.lambdify((s, w), A)
BB = sp.lambdify(b, B)
CC = sp.lambdify(c, C)
DD = sp.lambdify(d, D)

z = X.T.row_join(sp.Matrix([s])).row_join(sp.Matrix([w])).row_join(B.T).row_join(C).row_join(D).T
#print z

fx = A*X+B*U
f = sp.Matrix([0 for _ in range(len(z)-len(x))])
f = fx.col_join(f)

ff = sp.lambdify(z, f)

FF = sp.lambdify(z, f.jacobian(z))

h = C*X+D*U
hh = sp.lambdify(X.T.row_join(U), h)

HH = sp.lambdify(z.T.row_join(U), h.jacobian(z))


In [ ]:
rho = 0.1


R = np.eye(len(z))

#print np.diag([sym for sym in z])

q = np.array([])
for sym in z:
    if sym in X.T.row_join(C):
        q = np.hstack((q, 0))
    else:
        q = np.hstack((q, 1))

Q = rho * sp.Matrix(np.diag(q))
#Q

In [ ]:
#%time

# initial
# A
s_k0 = [-1] # real Teil
w_k0 = [1]  # (+/-) imag Teil

# b
b_k0 = [-2, 1]

# c
c_k0 = [0.1, 0.2]

# d
d_k0 = [0]


P_k0 = np.eye(len(z))

x_k0 = [0, 0]
u_k0 = [1]

T = 0.0001


# First values
x_k = x_k0
u_k = u_k0
z_k = x_k0 + s_k0 + w_k0 + b_k0 + c_k0 + d_k0
P_k = P_k0

# Loop start ?

##### Reconstruction from z
#print z_k
x_k = z_k[0:2]
#print x_k
s_k = z_k[2]
#print s_k
w_k = z_k[3]
#print w_k
b_k = z_k[4:6]
#print b_k
c_k = z_k[6:8]
#print c_k
d_k = [z_k[8]]
#print d_k

## System
A_k = AA(s_k, w_k)
B_k = BB(*b_k)
C_k = CC(*c_k)
D_k = DD(*d_k)


##### Evaluation
# State space
dx_k = A_k.dot(x_k) + B_k.dot(u_k)
y_k = C_k.dot(x_k) + D_k.dot(u_k)

# Concate values for lambdafied Jacobians
zu_k = z_k + u_k
xu_k = x_k + u_k

h_k = hh(*xu_k) # h
H_k = HH(*zu_k) # h.Jacobian

f_k = ff(*z_k) # f
F_k = FF(*z_k) #f.Jacobian


##### Kalman Filter

# Prediction
K_k = P_k*H_k.T * la.inv(H_k*P_k*H_k.T + R)
Ps_k = (np.eye(len(z)) - K_k*H_k)*P_k # P*_k

# Correction
P_k1 = Ps_k + T*(F_k*Ps_k + Ps_k*F_k.T + Q)
z_k1 = z_k + T*f_k + K_k*(y_k - h_k)

